worker ids:
46994197,CA,Redondo Beach, Salim, 21
47200615,NY,Ossining, Hee Dong, 21, 5
46545976,NY,Newburgh, Eva-Maria, 21, 3
46092070,NY,New City, Sara, 18, 2

In [ ]:
import pandas as pd
from datetime import datetime

name = {46994197:"Salim", 47200615:"Hee Dong", 46545976: "Eva-Maria", 46092070: "Sara"}
file_name = "/dccstor/srosent2/generative/appen/job_1988758NQLFQA.json"
data = pd.read_json(file_name, lines=True)

In [ ]:

skip_workers = {}
time_workers = {}
num_complete = {}

for worker in name:
    skip_workers[worker] = 0
    num_complete[worker] = 0
    # time_workers[worker] = 0

for index, row in data.iterrows():
    info = row['results']['judgments'][0]
    # print(row['results'])
        
    start = datetime.strptime(info['started_at'][11:-6],"%H:%M:%S")
    end = datetime.strptime(info['created_at'][11:-6],"%H:%M:%S")
    if info['worker_id'] not in time_workers:
        time_workers[info['worker_id']] = end-start
    else:
        time_workers[info['worker_id']] += end-start
    num_complete[info['worker_id']] += 1
    print(info['unit_data']['question_id']  + ": " + info['unit_data']['question'])
    print(name[info['worker_id']] + ": " + str(end-start))
    print("PARAGRAPH:")
    for sentence in info['unit_data']['long_answer']:
        print(sentence)
    if 'minimal_answer' in info['unit_data'] and info['unit_data']['minimal_answer'] is not None:
        print("minimal answer: " + info['unit_data']['minimal_answer'])
    if 'skipthere_is_no_answer' in info['data'] or 'there_is_no_answer' in info['data']:
        skip_workers[info['worker_id']] += 1
        print("SKIP")
    else:
        print("ANSWER: ")
        if 'paragraph_sentences' in info['data']:
            for sentence in info['data']['paragraph_sentences']:
                print(sentence)
        else:
            print("no sentences selected")
            print(info['data'])
        if 'type_your_answer_here_keep_your_answer_as_close_to_the_passage_as_possible_' not in info['data']:
        #    print(info['data']['type_your_answer_here_keep_your_answer_as_close_to_the_passage_as_possible_'])
        #else:
            print("no answer provided")
   
    print("---")
    # break
    # print(str(info['worker_id']) + "," + info['region'] + "," + info['city'] + "," + info['started_at'])
print(skip_workers)
for worker in name:
    print(name[worker] + ": " + str(time_workers[worker]/num_complete[worker]) + " skip: " +  str(skip_workers[worker]))

In [10]:
import pandas as pd
from datetime import datetime

name = {47200615:"Hee Dong", 46545976: "Eva Maria", 46092070: "Sara", 45676624: "Mohamed", 46373812: "Chie", 46545946: "Joekie", 46954475: "Arafat", 46994197: "Salim"}
# file_name = "/dccstor/srosent2/generative/appen/round1_jobs/job_2198088.json.zip"
# data1 = pd.read_json(file_name, lines=True)
# print(len(data1))
# file_name = "/dccstor/srosent2/generative/appen/round1_jobs/job_2203926.json.zip"
# data2 = pd.read_json(file_name, lines=True)
# print(len(data2))
# # data = pd.concat([data1, data2])
# # print(len(data))
# file_name = "/dccstor/srosent2/generative/appen/round1_jobs/job_2222665.json.zip"
# data3 = pd.read_json(file_name, lines=True)
# print(len(data3))
# file_name = "/dccstor/srosent2/generative/appen/round1_jobs/job_2244326.json.zip"
# data4 = pd.read_json(file_name, lines=True)
# print(len(data4))
# data = pd.concat([data1, data2, data3, data4])
file_name = "/dccstor/srosent2/generative/appen/round1_jobs/job_2261968.json.zip"
data = pd.read_json(file_name, lines=True)
print(len(data))
print(data.columns)
#data = data.sample(frac = 1)


89
Index(['id', 'data', 'judgments_count', 'state', 'agreement', 'missed_count',
       'gold_pool', 'answers_hidden', 'created_at', 'updated_at', 'job_id',
       'results'],
      dtype='object')


In [3]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def rougeL(answer1, answer2):
    scores = scorer.score(answer1, answer2)
    return scores

In [ ]:

def process_data(data):

    avg_rouge_all =  {"Hee Dong": [], "Eva Maria": [], "Sara": [], "Mohamed": [], "Chie": [], "Joekie": [], "Arafat": []} 
    partial =  {"Hee Dong": 0, "Eva Maria": 0, "Sara": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0, "Arafat": 0} 
    
    for index, row in data.iterrows():
        avg_rouge = {"Hee Dong": [], "Eva Maria": [], "Sara": [], "Mohamed": [], "Chie": [], "Joekie": [], "Arafat": []}
        print(str(row['data']['question_id']) + ": " + row['data']['question'])
        print("Title: " + str(row['data']['title']))
        for sentence in row['data']['long_answer']:
            print(sentence)
        print("---")
        judgements = row['results']['judgments']
        skip = set()
        scores = {}

        i = 0
        for i in range(len(judgements)):
            answer_type = judgements[i]['data']['how_would_you_describe_the_questionanswer']

            if answer_type == 'complete' or answer_type == 'partial':
                if answer_type == 'partial':
                    partial[name[judgements[i]['worker_id']]] += 1
                # if 'type_your_answer_here_keep_your_answer_as_close_to_the_passage_as_possible_' not in judgements[i]['data']:
                #     print("Error!")
                #     print(judgements[i])
                #     continue
                # type_your_answer_here_keep_your_answer_as_close_to_the_passage_as_possible_
                answer1 = judgements[i]['data']['type_your_answer_here_it_should_be_concise_and_only_come_from_the_passagetitle_']
                print(name[judgements[i]['worker_id']] + ": " + answer1 + " (" + answer_type + ")")
                if 'comments_optional' in judgements[i]['data']:
                    print("comment: " + judgements[i]['data']['comments_optional'])
            else:
                skip.add(name[judgements[i]['worker_id']] + ": " + answer_type)
                continue

            for j in range(i+1,len(judgements)):
                answer_type = judgements[j]['data']['how_would_you_describe_the_questionanswer']

                if answer_type == 'complete' or answer_type == 'partial':
                    answer2 = judgements[j]['data']['type_your_answer_here_it_should_be_concise_and_only_come_from_the_passagetitle_']
                    # print(str(judgements[j]['worker_id']) + ": " + answer)
                else:
                    continue
                score = rougeL(answer1, answer2)
                if name[judgements[i]['worker_id']] == "Arafat" or name[judgements[i]['worker_id']] == "Sara" \
                     or name[judgements[j]['worker_id']] == "Arafat" or name[judgements[j]['worker_id']] == "Sara":
                    scores[name[judgements[i]['worker_id']] + "-" + name[judgements[j]['worker_id']]] = score['rougeL'].fmeasure
                    avg_rouge[name[judgements[i]['worker_id']]].append(score['rougeL'].fmeasure)
                    avg_rouge[name[judgements[j]['worker_id']]].append(score['rougeL'].fmeasure)
        print("skip: " + str(skip))
        scores = sorted(scores.items(), key=lambda item: item[1], reverse=True)

        for score in scores:
            print(score)

        print("Average: ")
        for worker in avg_rouge:
            if len(avg_rouge[worker]) > 0:
                avg = sum(avg_rouge[worker])/len(avg_rouge[worker])
                print(worker + ": " + str(avg))
                avg_rouge_all[worker].append(avg)
        print("-----------------")

        if index > 20:
            break
    print("Annotator| Avg RougeL|# answered")
    print("-- | -- | --")
    for worker in avg_rouge_all:
        avg = sum(avg_rouge_all[worker])/len(avg_rouge_all[worker])
        print(worker + "|" + str(avg) + "|" + str(len(avg_rouge_all[worker])))
        # print("partial count: " + worker + ": " + str(partial[worker]))

process_data(data)

This next code snippet is to analyze data when there is one annotation per example

In [9]:
import pandas
import numpy as np

def process_single_data(data):

    annotation_count =  {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0, "Salim": 0} 
    skip_count =  {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0, "Salim": 0} 
    sentence_count = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0, "Salim": 0} 
    sentences_per_paragraph = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0, "Salim": 0} 
    num_with_minimal_answer = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0, "Salim": 0} 
    non_consecutive_count = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0, "Salim": 0} 
    answer_len = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0, "Salim": 0} 
    avg_length = 0
    avg_noncons_length = 0
    noncons_count = 0
    total_answered = 0
    # csvfile = open('/dccstor/srosent2/generative/appen/job_2035917.non_consecutive.xlsx', 'w', newline='')
    all_output = []

    for index, row in data.iterrows():
        judgement = row['results']['judgments'][0]

        # if len(row['data']['long_answer']) < 5:
        #     continue

        if judgement['unit_data']['minimal_text'] != '':
            num_with_minimal_answer[name[judgement['worker_id']]] += 1
            # continue
        
        answer_type = judgement['data']['how_would_you_describe_the_questionanswer']
        sentences_per_paragraph[name[judgement['worker_id']]] += len(row['data']['long_answer'])

        if answer_type == 'complete' or answer_type == 'partial':

            if 'type_your_answer_here_it_should_be_concise_and_only_come_from_the_passagetitle_' not in judgement['data']:
                print(judgement)
                continue
            # if answer_type == 'partial':
            #     partial[name[judgement['worker_id']]] += 1
            if 'paragraph_sentences' in judgement['data']:
                sentence_count[name[judgement['worker_id']]] += len(judgement['data']['paragraph_sentences'])
            answer1 = judgement['data']['type_your_answer_here_it_should_be_concise_and_only_come_from_the_passagetitle_']
            avg_length += len(answer1.split(" "))
            total_answered += 1
            answer_len[name[judgement['worker_id']]] += len(answer1.split(" "))

            last_consecutive = -1
            sent_i = 0
            non_consecutive = False
            for sentence in row['data']['long_answer']:
                if 'paragraph_sentences' in judgement['data'] and sentence in judgement['data']['paragraph_sentences']:
                    if last_consecutive != -1 and last_consecutive != sent_i-1:
                        non_consecutive = True
                        break
                    last_consecutive = sent_i
                sent_i += 1  

            if non_consecutive:
                avg_noncons_length += len(answer1.split(" "))
                noncons_count += 1
                output = [str(annotation_count[name[judgement['worker_id']]]) , str(row['data']['question_id']) , row['data']['question'], str(row['data']['title'])]
                
                sentences = ""
                for sentence in row['data']['long_answer']:
                    if 'paragraph_sentences' in judgement['data'] and sentence in judgement['data']['paragraph_sentences']:
                        sentences += "SELECTED " + sentence + "\n"
                    else:
                        sentences += sentence + "\n"
                output.append(sentences)
                output.append(name[judgement['worker_id']])
                output.append(answer1)
                output.append(answer_type)
                if 'comments_optional' in judgement['data']:
                    output.append(judgement['data']['comments_optional'])
                all_output.append(output)
                non_consecutive_count[name[judgement['worker_id']]] += 1
            annotation_count[name[judgement['worker_id']]] += 1
        else:
            skip_count[name[judgement['worker_id']]] += 1
            continue
    print(len(all_output))
    print("avg length")
    print(avg_length/total_answered)
    print("avg non cons. length")
    print(avg_noncons_length/noncons_count)
    # with pd.ExcelWriter("/dccstor/srosent2/generative/appen/full_2.6.23.non_consecutive.xlsx", engine='xlsxwriter') as writer:
    #     df = pd.DataFrame(all_output)
    #     df.to_excel(writer, sheet_name='Sheet1')
    #     workbook  = writer.book
    #     worksheet = writer.sheets['Sheet1']
    #     cell_format = workbook.add_format({'text_wrap': True})
    #     worksheet.set_column('A:Z', cell_format=cell_format)
        
    print("Annotator | # completed | # answered | # skipped | # non consecutive | # with minimal answer | # sentences selected |  # sentences per answer | # sentences per paragraph")
    print("-- | -- | -- | -- | -- | -- | -- | -- | --")
    for worker in annotation_count:
        print(worker + "|" + str(annotation_count[worker] + skip_count[worker]) +  "|" +
          str(annotation_count[worker]) + "|" + str(skip_count[worker]) + "|" + str(non_consecutive_count[worker]) + "|" + str(num_with_minimal_answer[worker]) + "|" + str(sentence_count[worker]) + "|" +
          str(round(sentence_count[worker]/(annotation_count[worker]-skip_count[worker]),2)) + "|" +  str(round(sentences_per_paragraph[worker]/(annotation_count[worker]-skip_count[worker]),2)) + "|" + str(answer_len[worker]/annotation_count[worker]))
        # print("partial count: " + worker + ": " + str(partial[worker]))

print(len(data))
process_single_data(data)

3818
{'id': 6549612615, 'created_at': '2022-12-09T04:10:15+00:00', 'started_at': '2022-12-09T04:04:02+00:00', 'acknowledged_at': None, 'external_type': 'cf_internal', 'golden': False, 'missed': None, 'rejected': None, 'tainted': False, 'country': 'US', 'region': 'NY', 'city': 'Levittown', 'unit_id': 3382327340, 'job_id': 2022794, 'worker_id': 45676624, 'trust': 1.0, 'worker_trust': 0.9549874458874461, 'unit_state': 'finalized', 'data': {'how_would_you_describe_the_questionanswer': 'partial'}, 'unit_data': {'question_id': '8993854018546607757', 'question': 'at the end of a normal exhalation the diaphragm is', 'title': 'Exhalation', 'url': 'https://en.wikipedia.org//w/index.php?title=Exhalation&amp;oldid=843612025', 'long_answer': ['The main reason for exhalation is to rid the body of carbon dioxide , which is the waste product of gas exchange in humans .', 'Air is brought in the body through inhalation .', 'During this process air is taken in through the lungs .', 'Diffusion in the alve

2095
{'id': 6549612615, 'created_at': '2022-12-09T04:10:15+00:00', 'started_at': '2022-12-09T04:04:02+00:00', 'acknowledged_at': None, 'external_type': 'cf_internal', 'golden': False, 'missed': None, 'rejected': None, 'tainted': False, 'country': 'US', 'region': 'NY', 'city': 'Levittown', 'unit_id': 3382327340, 'job_id': 2022794, 'worker_id': 45676624, 'trust': 1.0, 'worker_trust': 0.9549874458874461, 'unit_state': 'finalized', 'data': {'how_would_you_describe_the_questionanswer': 'partial'}, 'unit_data': {'question_id': '8993854018546607757', 'question': 'at the end of a normal exhalation the diaphragm is', 'title': 'Exhalation', 'url': 'https://en.wikipedia.org//w/index.php?title=Exhalation&amp;oldid=843612025', 'long_answer': ['The main reason for exhalation is to rid the body of carbon dioxide , which is the waste product of gas exchange in humans .', 'Air is brought in the body through inhalation .', 'During this process air is taken in through the lungs .', 'Diffusion in the alveoli allows for the exchange of O into the pulmonary capillaries and the removal of CO and other gases from the pulmonary capillaries to be exhaled .', 'In order for the lungs to expel air the diaphragm relaxes , which pushes up on the lungs .', 'The air then flows through the trachea then through the larynx and pharynx to the nasal cavity and oral cavity where it is expelled out of the body .', 'Exhalation takes longer than inhalation since it is believed to facilitate better exchange of gases .', 'Parts of the nervous system help to regulate respiration in humans .', "The exhaled air is n't just carbon dioxide ; it contains a mixture of other gases .", 'Human breath contains volatile organic compounds ( VOCs ) .', 'These compounds consist of methanol , isoprene , acetone , ethanol and other alcohols .', 'The exhaled mixture also contains ketones , water and other hydrocarbons .'], 'minimal_text': ''}}
307
Annotator | # completed | # answered | # skipped | # non consecutive | # with minimal answer | # sentences selected |  # sentences per answer | # sentences per paragraph
-- | -- | -- | -- | -- | -- | -- | -- | --
Hee Dong|600|452|148|71|112|790|2.6|10.25
Eva Maria|569|490|79|57|76|657|1.6|8.09
Mohamed|143|109|34|27|10|242|3.23|12.37
Chie|608|523|85|114|57|1235|2.82|8.41
Joekie|160|137|23|35|19|260|2.28|8.31
Salim|14|12|2|3|9|14|1.4|6.0

In [12]:
# load the annotations that have already been completed in round 2 setup so they don't get completed again. 
# keep track of the IDs
import glob

dir_name = "/dccstor/srosent2/generative/appen/round2_1.20.2023/job_2035917.round2.*.csv"
files = glob.glob(dir_name)
dir_name = "/dccstor/srosent2/generative/appen/full_2.6.23.round2/*.csv"
files.extend(glob.glob(dir_name))
data_ids = []
for file_name in files:
    seen_data = pd.read_csv(file_name)
    data_ids.extend(list(seen_data['id']))
print(data_ids)

[2495396943532822225, -229082287897515617, -2264134242434198326, 2079100088522925638, 2980592935468943450, 6161497846937596390, -4959346488900267637, -2465861575714299171, -2189548918642725127, 7098636154491383343, -3933208289628918028, -1257580038851946042, 5121004319758778839, -1399826316214371736, -6946841282838040550, 8427201419256600035, -1584181447791057413, -4084391713873607156, 8777897555648878475, 5808798670950006232, 7898252756426188619, -936565044218557383, -4618864426037979762, 8116928765126546687, -1729923543484435760, -1062046011613182635, -4209352084651040508, 7483313621841077081, 996768280100184490, -8619899755897217555, 923449576656273005, -1275524742182878778, 873564346673136599, -779483661367421641, 1688288076551153504, -1157343340377926175, 662120736332570642, 5866872638910116716, 3998102536607710960, -2951084596378848197, 3037366949322470211, 8713188615749424843, 3103906544916591053, 4947463880421396368, 5461188725342525320, -2705463144889337080, -13718948346650563

In [11]:
import pandas as pd
import numpy as np
import random

name = {47200615:"Hee Dong", 46545976: "Eva Maria", 46092070: "Sara", 45676624: "Mohamed", 46373812: "Chie", 46545946: "Joekie", 46954475: "Arafat", 46994197: "Salim"}

def appen_round2_check(data, seen_ids=None):

    annotation_count =  {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    skip_count =  {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    sentence_count = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    sentences_per_paragraph = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    num_with_minimal_answer = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    new_task = {"Hee Dong": [], "Chie": []} #, "Joekie": []}
    annotators = ["Hee Dong", "Chie"] #, "Joekie"]
    
    all_output = []
    seen_id_count = 0

    for index, row in data.iterrows():

        if seen_ids is not None and int(row['data']['question_id']) in seen_ids:
            seen_id_count += 1
            continue
        judgement = row['results']['judgments'][0]

        if judgement['unit_data']['minimal_text'] != '' and name[judgement['worker_id']] in annotators:
            num_with_minimal_answer[name[judgement['worker_id']]] += 1
            # continue
        
        answer_type = judgement['data']['how_would_you_describe_the_questionanswer']
        if name[judgement['worker_id']] in annotators:
            sentences_per_paragraph[name[judgement['worker_id']]] += len(row['data']['long_answer'])

        if answer_type == 'complete' or answer_type == 'partial':

            if 'type_your_answer_here_it_should_be_concise_and_only_come_from_the_passagetitle_' not in judgement['data']:
                print(judgement)
                continue
            # if answer_type == 'partial':
            #     partial[name[judgement['worker_id']]] += 1
            if 'paragraph_sentences' in judgement['data'] and name[judgement['worker_id']] in annotators:
                sentence_count[name[judgement['worker_id']]] += len(judgement['data']['paragraph_sentences'])
            answer1 = judgement['data']['type_your_answer_here_it_should_be_concise_and_only_come_from_the_passagetitle_']

            last_consecutive = -1
            sent_i = 0
            non_consecutive = False
            for sentence in row['data']['long_answer']:
                if 'paragraph_sentences' in judgement['data'] and sentence in judgement['data']['paragraph_sentences']:
                    if last_consecutive != -1 and last_consecutive != sent_i-1:
                        non_consecutive = True
                        break
                    last_consecutive = sent_i
                sent_i += 1  

            if non_consecutive:
                output = [str(row['data']['question_id']) , row['data']['question'], str(row['data']['title'])]
                
                sentences = ""
                selected = ""
                for sentence in row['data']['long_answer']:
                    sentences += sentence + "&nbsp;"
                    if 'paragraph_sentences' in judgement['data'] and sentence in judgement['data']['paragraph_sentences']:
                        selected += "True"
                    else:
                        selected += "False"
                    selected += "&nbsp;"
                title_used_in_answer = False
                if 'title_used_in_answer' in judgement['data']:
                    title_used_in_answer = True
                output.append(sentences)
                output.append(selected)
                output.append(title_used_in_answer)
                output.append(name[judgement['worker_id']])
                output.append(answer1)

                random_person = annotators[random.randint(0,len(annotators)-1)]
                
                while random_person == name[judgement['worker_id']] or len(new_task[random_person]) > 15:
                    random_person = annotators[random.randint(0,len(annotators)-1)]
                # print(random_person)
                new_task[random_person].append(output)
                if name[judgement['worker_id']] in annotators:
                    annotation_count[name[judgement['worker_id']]] += 1
        else:
            if name[judgement['worker_id']] in annotators:
                skip_count[name[judgement['worker_id']]] += 1
            continue

    for person in new_task:
        df = pd.DataFrame(new_task[person])
        print(person + ": " + str(len(new_task[person])))
        df.to_csv("/dccstor/srosent2/generative/appen/round2_jobs6/input/job_round2." + person + ".csv", encoding='utf-8', header=["id","question","title","long_answer","selected","title_used","worker","answer"])
    print(seen_id_count)
    # print("Annotator | # completed | # answered | # skipped | # with minimal answer | # sentences selected |  # sentences per answer | # sentences per paragraph")
    # print("-- | -- | -- | -- | -- | -- | -- | -- ")
    # for worker in annotation_count:
    #     print(worker + "|" + str(annotation_count[worker]) +  "|" +
    #       str(annotation_count[worker] - skip_count[worker]) + "|" + str(skip_count[worker]) + "|" + str(num_with_minimal_answer[worker]) + "|" + str(sentence_count[worker]) + "|" +
    #       str(round(sentence_count[worker]/(annotation_count[worker]-skip_count[worker]),2)) + "|" +  str(round(sentences_per_paragraph[worker]/(annotation_count[worker]-skip_count[worker]),2)))
    #     # print("partial count: " + worker + ": " + str(partial[worker]))

# file_name = "/dccstor/srosent2/generative/appen/round1_jobs/job_2145426.json.zip"
# data = pd.read_json(file_name, lines=True)

appen_round2_check(data, seen_ids=None)

{'id': 6681520194, 'created_at': '2023-09-06T11:51:13+00:00', 'started_at': '2023-09-06T11:49:13+00:00', 'acknowledged_at': None, 'external_type': 'cf_internal', 'golden': False, 'missed': None, 'rejected': None, 'tainted': False, 'country': 'US', 'region': 'New York', 'city': 'New Windsor', 'unit_id': 3493405941, 'job_id': 2261968, 'worker_id': 46545976, 'trust': 1.0, 'worker_trust': 0.9865308641975301, 'unit_state': 'finalized', 'data': {'how_would_you_describe_the_questionanswer': 'complete', 'please_confirm_the_following_before_proceeding_if_you_are_not_skipping_this_question': ['reread']}, 'unit_data': {'question_id': '5092896004240326569', 'question': 'can smooth muscle change the diameter of blood vessels', 'title': 'Vascular smooth muscle', 'url': 'https://en.wikipedia.org//w/index.php?title=Vascular_smooth_muscle&amp;oldid=812798116', 'long_answer': ['α 1 ( \\ displaystyle \\ alpha _', '( 1 ) ) receptors .', 'Under NE binding α 1 ( \\ displaystyle \\ alpha _', '( 1 ) ) recepto

In [9]:
# Get ones that were marked as having no answer

import pandas as pd
import numpy as np
import random
import glob

name = {47200615:"Hee Dong", 46545976: "Eva Maria", 46092070: "Sara", 45676624: "Mohamed", 46373812: "Chie", 46545946: "Joekie", 46954475: "Arafat", 46994197: "Salim"}

def appen_round2_noanswer(data, seen_ids=None):

    annotation_count =  {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    skip_count =  {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    sentence_count = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    sentences_per_paragraph = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    num_with_minimal_answer = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    new_task = {"Hee Dong": [], "Eva Maria": [], "Chie": [], "Joekie": []}
    annotators = ["Hee Dong", "Eva Maria", "Mohamed", "Chie", "Joekie"]
    
    all_output = []
    seen_id_count = 0

    for index, row in data.iterrows():

        if seen_ids is not None and int(row['data']['question_id']) in seen_ids:
            seen_id_count += 1
            continue
        judgement = row['results']['judgments'][0]

        if judgement['unit_data']['minimal_text'] != '' and name[judgement['worker_id']] in annotators:
            num_with_minimal_answer[name[judgement['worker_id']]] += 1
            # continue
        
        answer_type = judgement['data']['how_would_you_describe_the_questionanswer']
        if name[judgement['worker_id']] in annotators:
            sentences_per_paragraph[name[judgement['worker_id']]] += len(row['data']['long_answer'])

        if answer_type == 'complete' or answer_type == 'partial':
            continue
        else:
            if name[judgement['worker_id']] in annotators:
                skip_count[name[judgement['worker_id']]] += 1
            output = [str(row['data']['question_id']) , row['data']['question'], str(row['data']['title'])]
                
            sentences = ""
            for sentence in row['data']['long_answer']:
                sentences += sentence + "&nbsp;"
            output.append(sentences)
            output.append(name[judgement['worker_id']])
            output.append(answer_type)

            random_person = annotators[random.randint(0,len(annotators)-1)]
            while random_person == name[judgement['worker_id']] or random_person == "Mohamed" or len(new_task[random_person]) > 51:
                random_person = annotators[random.randint(0,len(annotators)-1)]
            print(random_person)
            new_task[random_person].append(output)
            if name[judgement['worker_id']] in annotators:
                annotation_count[name[judgement['worker_id']]] += 1

    for person in new_task:
        df = pd.DataFrame(new_task[person])
        print(person + ": " + str(len(new_task[person])))
        df.to_csv("/dccstor/srosent2/generative/appen/no_answer_round2/job_roundNA." + person + ".csv", encoding='utf-8', header=["id","question","title","long_answer","worker","answer_type"])
    print(seen_id_count)

paths = glob.glob("/dccstor/srosent2/generative/appen/round1_jobs/job_*.json.zip")

for file_name in paths:
    data = pd.read_json(file_name, lines=True)
    data.append(data, ignore_index=True)
# data = pd.DataFrame([pd.read_json(p, typ="series", lines=True) for p in paths])

appen_round2_noanswer(data, seen_ids=None)

/tmp/ipykernel_900061/4117677936.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data.append(data, ignore_index=True)
/tmp/ipykernel_900061/4117677936.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data.append(data, ignore_index=True)
/tmp/ipykernel_900061/4117677936.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data.append(data, ignore_index=True)
/tmp/ipykernel_900061/4117677936.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data.append(data, ignore_index=True)
/tmp/ipykernel_900061/4117677936.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pa

Hee Dong
Chie
Chie
Chie
Chie
Chie
Chie
Chie
Chie
Chie
Hee Dong
Chie
Chie
Chie
Hee Dong
Hee Dong
Chie
Chie
Hee Dong
Hee Dong
Chie
Chie
Chie
Chie
Chie
Chie
Hee Dong
Hee Dong
Hee Dong
Chie
Chie
Chie
Chie
Chie
Chie
Chie
Chie
Chie
Chie
Chie
Chie
Chie
Hee Dong
Chie
Hee Dong
Chie
Chie
Chie
Chie
Chie
Chie
Hee Dong
Chie
Chie
Chie
Chie
Chie
Hee Dong
Chie
Chie
Chie
Hee Dong
Chie
Hee Dong
Chie
Chie
Hee Dong
Chie
Chie
Hee Dong
Chie
Chie
Chie
Hee Dong
Chie
Hee Dong
Chie
Chie
Chie
Chie
Hee Dong
Chie
Hee Dong
Chie
Hee Dong
Chie
Chie
Chie
Hee Dong
Chie
Hee Dong
Hee Dong
Chie
Chie
Chie
Chie
Chie
Chie
Chie
Hee Dong
Chie
Hee Dong
Chie
Hee Dong
Chie
Hee Dong
Hee Dong
Hee Dong
Chie
Hee Dong
Hee Dong
Chie
Chie
Hee Dong
Chie
Chie
Hee Dong
Chie
Chie
Chie
Chie
Chie
Chie
Chie
Chie
Chie
Chie
Hee Dong
Chie
Hee Dong
Chie
Chie
Hee Dong
Chie
Chie
Chie
Hee Dong
Hee Dong
Hee Dong
Chie
Hee Dong
Hee Dong
Chie
Chie
Chie
Chie
Chie
Hee Dong
Chie
Chie
Hee Dong
Chie
Chie
Hee Dong
Chie
Hee Dong
Hee Dong
Chie
Chie
Chie
Hee Dong

/tmp/ipykernel_900061/4117677936.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data.append(data, ignore_index=True)


In [3]:
import pandas as pd
from datetime import datetime
import os

name = {47200615:"Hee Dong", 46545976: "Eva Maria", 45676624: "Mohamed", 46373812: "Chie", 46545946: "Joekie"}
dir_name = "/dccstor/srosent2/generative/appen/Round2/"
files = os.listdir(dir_name)

new_annotations = None
gold_annotations = None

for file_name in files:
    if file_name.endswith('json.zip'):
        if new_annotations is None:
            new_annotations = pd.read_json(dir_name + file_name, lines=True)
        else:
            new_annotations = new_annotations.append(pd.read_json(dir_name + file_name, lines=True))
    elif file_name.endswith('csv'):
        gold_annotations = pd.read_csv(dir_name + file_name, delimiter=',')

In [4]:

improvement =  {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0}
needs_improvement =  {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0}

all_output = []
for index, row in new_annotations.iterrows():
    output = []
    print(row['data']['question'])
    output.append(row['data']['id'])
    output.append(row['data']['question'])
    gold_annotation = gold_annotations[gold_annotations['ID'] == int(row['data']['id'])]

    sentences = ""
    judgement = row['results']['judgments'][0]
    input = row['data']
    for sentence in row['data']['long_answer']:
        selected = []
        if 'long_answer' in input and input['selected'][input['long_answer'].index(sentence)] == 'True':
            selected.append("1")
        if 'paragraph_sentences' in judgement['data'] and sentence in judgement['data']['paragraph_sentences']:
            selected.append("2")
        if len(selected) > 0:
            sentences += "SELECTED " + str(selected) + ": "
        sentences += sentence + "\n"

    answer1 = row['data']['answer']
    output.append(sentences)
    output.append(row['data']['worker'])
    output.append(answer1)
    print(row['data']['worker'] + ": " + answer1)
    output.append(name[row['results']['judgments'][0]['worker_id']])
    output.append(row['results']['judgments'][0]['data']['did_you_need_to_edit_the_answer'])
    if 'update_the_answer_here_if_needed' in row['results']['judgments'][0]['data']:
        answer2 = row['results']['judgments'][0]['data']['update_the_answer_here_if_needed']
        print(name[row['results']['judgments'][0]['worker_id']] + ": " + answer2)
        if 'did_you_need_to_edit_the_answer' in row['results']['judgments'][0]['data']:
            print(row['results']['judgments'][0]['data']['did_you_need_to_edit_the_answer'])
        rouge1 = rougeL(answer1, answer2)['rougeL'].fmeasure
        print(row['data']['worker'] + "/" + name[row['results']['judgments'][0]['worker_id']] + ": " + str(rouge1))
        output.append(answer2)
        output.append(rouge1)
    else:
        output.append("")
        output.append("")
    gold_answer = gold_annotation['Rewritten Answer'].iloc[0]
    output.append(gold_annotation['Assigned To'].iloc[0])
    if isinstance(gold_answer, str):
        needs_improvement[name[row['results']['judgments'][0]['worker_id']]] += 1
        print(gold_annotation['Assigned To'].iloc[0] + ": " + gold_answer)
        rouge2 = rougeL(answer2, gold_answer)['rougeL'].fmeasure
        rouge3 = rougeL(answer1, gold_answer)['rougeL'].fmeasure
        output.append(gold_answer)
        output.append(rouge2)
        output.append(rouge3)
        # if rouge3 > rouge2:
        improvement[name[row['results']['judgments'][0]['worker_id']]] += rouge3 - rouge2
        if rouge3 -.2 < rouge2:
            print("****")
        print(gold_annotation['Assigned To'].iloc[0] + "/" + row['data']['worker'] + ": " + str(rouge2))
        print(gold_annotation['Assigned To'].iloc[0] + "/" + name[row['results']['judgments'][0]['worker_id']] + ": " + str(rouge3))
    else:
        output.append("")
        output.append("")
        output.append("")
    print('--')
    all_output.append(output)

for annotator in improvement:
    if improvement[annotator] != 0:
        print(annotator + ": " + str(improvement[annotator]/needs_improvement[annotator]))
    else:
        print(annotator + ": 0/" + str(needs_improvement[annotator]))

with pd.ExcelWriter("/dccstor/srosent2/generative/appen/job_2035917.non_consecutive_followup_round2.xlsx", engine='xlsxwriter') as writer:
    df = pd.DataFrame(all_output)
    df.to_excel(writer, sheet_name='Sheet1')
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']
    cell_format = workbook.add_format({'text_wrap': True})
    worksheet.set_column('A:Z', cell_format=cell_format)

which pigment determines the colour of human hair
Mohamed: Eumelanin, which is behind the colors black , blond , and brown, and Pheomelanin, which is behind the red color, are the two pigments that determine the human hair color.
Joekie: Eumelanin determines the hair colors gray, black, blond and brown, and Pheomelanin, determines red hair color.
['extra info']
Mohamed/Joekie: 0.5454545454545454
--
when did we start putting shoes on horses
Hee Dong: Historians differ on the origin of the horseshoe. Because iron was a valuable commodity, and any worn out items were generally reforged and reused , it is difficult to locate clear archaeological evidence. In 1897 four bronze horseshoes with what are apparently nail holes were found in an Etruscan tomb dated around 400 B.C. The assertion by some historians that the Romans invented the `` mule shoes '' sometime after 100 BC is supported by a reference by Catullus who died in 54 BC.
Joekie: Historians differ on the origin of the horseshoe. Be